In [1]:
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv

In [2]:
def get_states(n):
    return (range(0,n+1))

In [3]:
def get_transition_A(n):
    P = np.zeros((n+1,n+1))
    P[0,0] = 1
    P[n,n] = 1
    for i in range(1,n):
        P[i,i+1] = (n-i)/n
        P[i,i-1] = i/n
    return(P) 

In [4]:
def get_transition_B(n):
    P = np.zeros((n+1,n+1))
    P[0,0] = 1
    P[n,n] = 1
    for i in range(1,n):
        for j in range(0,n+1):
            if i != j:
                P[i,j] = 1/n
    return(P)

In [5]:
def get_rewards_A(n):
    R = np.zeros(n+1)
    R[0] = -1
    R[1] = -1/n
    R[n-1] = 1/n
    R[n] = 1
    return(R)

In [6]:
def get_rewards_B(n):
    R = np.zeros(n+1)
    R[0] = -1
    R[n] = 1
    return(R)

In [7]:
def update_state_val(n, gamma, v_old, transition_A, transition_B, rewards_A, rewards_B):
    v_new = np.array([max(rewards_A[i] + gamma*np.dot(transition_A[i,:],v_old), 
                          rewards_B[i] + gamma*np.dot(transition_B[i,:],v_old)) for i in range(0,n+1)])
    return(v_new)

In [8]:
def value_iteration(n_iter, n, gamma, v_0, transition_A, transition_B, rewards_A, rewards_B):
    for k in range(0,n_iter):
        v_0 = update_state_val(n, gamma, v_0, transition_A, transition_B, rewards_A, rewards_B)
    return(v_0)

In [9]:
def get_policy(n, gamma, v, transition_A, transition_B, rewards_A, rewards_B):
    policy = []
    for i in range(0,n+1):
        q_A = rewards_A[i] + gamma*np.dot(transition_A[i,:],v)
        q_B = rewards_B[i] + gamma*np.dot(transition_B[i,:],v)
        policy.append('A') if q_A > q_B else policy.append('B')
    return policy

In [10]:
def get_transition_policy(n, transition_A, transition_B, policy):
    transition_policy = np.zeros((n+1, n+1))
    for i in range(0,n+1):
        transition_policy[i,:] = transition_A[i,:] if policy[i] == 'A' else transition_B[i,:]
    return(transition_policy)

In [11]:
def probability_distribution(n, transition_policy):
    
    #We reorder our transition matrix to 
    #isolate transient states from absorbant states
    transition_policy = np.copy(transition_policy)
    transition_policy = np.roll(transition_policy,-1,axis=0)
    transition_policy = np.roll(transition_policy,-1,axis=1)
        
    Q = transition_policy[0:-2,0:-2]
    R = transition_policy[0:-2,-2:]
    N = inv(np.eye(n-1)-Q)
    
    result = np.dot(N,R)
    
    return(result[:,0])
    

In [12]:
def plot_result(n, result, policy):
        X = range(1,n)
        Y = result
        plt.figure()
        plt.bar(X,Y, width=0.5)
        for i in range(1, n):
            plt.annotate(policy[i], xy=(X[i-1],Y[i-1]))
        fig_name = 'figure' + str(n) + '.png'
        plt.savefig(fig_name)
        plt.show()

In [13]:
def MDP(n, gamma, n_iter):
    
    # ---- Building MDP ----
    S = get_states(n)
    transition_A = get_transition_A(n)
    transition_B = get_transition_B(n)
    rewards_A = get_rewards_A(n)
    rewards_B = get_rewards_B(n)
    # ----------------------

    # ---- Solving MDP -----
    v_0 = np.zeros(n+1)
    v_0[0] = -1
    v_0[n] = 1

    v = value_iteration(n_iter, n, gamma, v_0, transition_A, transition_B, rewards_A, rewards_B)
    policy = get_policy(n, gamma, v, transition_A, transition_B, rewards_A, rewards_B)
    transition_policy = get_transition_policy(n, transition_A, transition_B, policy)
    # ----------------------

    # ---- Plot results -----
    result = probability_distribution(n, transition_policy)
    #plot_result(n, result, policy) - Commented for PDF export
    # -----------------------
    

In [14]:
MDP(3, 0.8, 1000)

In [15]:
MDP(10,0.8,1000)

In [16]:
MDP(25,0.8,1000)